# Fronteira eficiente de Markovitz

Harry Markovitz criou em 1952 uma teoria de análise do risco de uma carteira de investimento. O objetivo do modelo é criar um portfólio que otimiza o risco-retorno dos investimentos disponíveis, escolhendo os pesos ideais para cada ativo na carteira.

## 1.0 Bibliotecas

In [3]:
import numpy                as np
import datetime             as dt
import pandas               as pd
import matplotlib.pyplot    as plt
import matplotlib.ticker    as mtick

from scipy.optimize         import minimize
from pandas_datareader      import data as pdr

# 2. Período de Análise

In [ ]:
inicio = dt.date(2015, 1, 1)
fim = dt.date(2023, 4, 30)